In [22]:
import os
import pandas as pd
import json

dataset_dir = os.path.expanduser('~') + os.sep + 'Kaggle' + os.sep + 'turkeyCompetition' + os.sep + 'dataset' + os.sep
train_filename = 'train.json'
test_filename = 'test.json'

In [36]:
#Look into a couple of samples from TRAIN dataset
train_data = []

with open(dataset_dir + train_filename) as f:
    train_data = json.load(f)
    
for idx, sample in enumerate(train_data):
    print("Keys available: {}".format(sample.keys()))
    for key in sample.keys():
        print("Key: {} -- Value: {}".format(key, sample[key]))
    print('\n')
    if idx == 2:
        break

Keys available: dict_keys(['vid_id', 'audio_embedding', 'start_time_seconds_youtube_clip', 'is_turkey', 'end_time_seconds_youtube_clip'])
Key: vid_id -- Value: kDCk3hLIVXo
Key: audio_embedding -- Value: [[172, 34, 216, 110, 208, 46, 95, 66, 161, 125, 83, 39, 106, 146, 90, 144, 109, 111, 194, 58, 24, 228, 81, 128, 18, 94, 197, 161, 56, 0, 85, 97, 206, 74, 165, 126, 95, 121, 132, 105, 105, 177, 16, 159, 160, 112, 92, 9, 191, 255, 211, 0, 76, 87, 24, 44, 122, 38, 183, 208, 200, 29, 81, 86, 212, 255, 241, 93, 255, 176, 52, 99, 132, 235, 59, 83, 73, 74, 124, 167, 160, 208, 136, 208, 255, 73, 147, 255, 53, 0, 156, 215, 255, 255, 68, 255, 148, 0, 247, 206, 91, 18, 177, 191, 54, 116, 79, 92, 36, 12, 153, 29, 96, 170, 26, 255, 162, 255, 0, 0, 1, 170, 0, 8, 255, 129, 4, 255], [171, 39, 199, 121, 238, 62, 59, 61, 170, 146, 96, 76, 121, 139, 76, 102, 146, 142, 159, 64, 0, 252, 119, 134, 93, 90, 176, 215, 120, 42, 134, 160, 159, 87, 86, 190, 111, 89, 61, 47, 142, 149, 25, 145, 177, 169, 116, 35, 90

In [37]:
#Look into a couple of samples from TEST dataset
test_data = []

with open(dataset_dir + test_filename) as f:
    test_data = json.load(f)
    
for idx, sample in enumerate(test_data):
    print("Keys available: {}".format(sample.keys()))
    for key in sample.keys():
        print("Key: {} -- Value: {}".format(key, sample[key]))
    print('\n')
    if idx == 2:
        break

Keys available: dict_keys(['start_time_seconds_youtube_clip', 'audio_embedding', 'vid_id', 'end_time_seconds_youtube_clip'])
Key: start_time_seconds_youtube_clip -- Value: 0
Key: audio_embedding -- Value: [[177, 20, 226, 132, 198, 81, 111, 59, 132, 184, 58, 43, 108, 85, 56, 88, 103, 194, 193, 53, 90, 255, 86, 70, 6, 109, 0, 242, 7, 2, 206, 108, 0, 91, 255, 180, 92, 20, 8, 218, 41, 42, 142, 255, 255, 65, 88, 25, 39, 229, 144, 0, 196, 129, 126, 123, 0, 76, 89, 161, 119, 178, 90, 127, 234, 202, 255, 69, 71, 0, 229, 47, 230, 166, 140, 76, 255, 255, 35, 255, 13, 178, 255, 188, 255, 0, 238, 203, 36, 48, 0, 255, 164, 186, 147, 255, 0, 50, 255, 188, 147, 255, 209, 217, 235, 232, 41, 200, 208, 196, 0, 192, 113, 210, 55, 132, 0, 255, 3, 255, 0, 255, 229, 0, 255, 184, 48, 255], [168, 19, 221, 138, 219, 88, 161, 77, 120, 182, 45, 82, 101, 100, 92, 87, 79, 202, 184, 60, 76, 255, 105, 49, 0, 143, 0, 155, 67, 15, 201, 64, 37, 55, 232, 191, 85, 0, 72, 218, 9, 21, 95, 227, 211, 93, 61, 132, 120, 175, 2

In [32]:
#Check if any clips have more than 10 samples
max_value = 0

for val in train_data:
    if max_value < len(val['audio_embedding']):
        max_value = len(val['audio_embedding'])
        
print(max_value)

10


In [ ]:
#Create the 

In [34]:
#Build neural network to start processing inputs
import tensorflow as tf

In [ ]:
def get_inputs():
    """
    Create TF Placeholders for input, targets, and learning rate.
    :return: Tuple (input, targets, learning rate)
    """
    tf_input = tf.placeholder(tf.int32, [None, None], name='input')
    tf_targets = tf.placeholder(tf.int32, [None], name='targets')
    tf_learning_rate = tf.placeholder(tf.float32, name='learning_rate')
    
    return tf_input, tf_targets, tf_learning_rate

